<p>Copyright by Aries Li</p>

In [1]:
import json
import os
from sutime import SUTime
import pdftotext
import glob
import nltk
import numpy as np
import re
import pandas as pd

In [3]:
with open('500 Account Statements Extracted Data.json', 'rb') as f:
    extracted_data = f.read()

In [4]:
extracted_data = json.loads(extracted_data)

In [5]:
jar_files = '/Users/aries/Desktop/research/stanford-corenlp-full-2018-10-05/jars/jars'
sutime = SUTime(jars=jar_files, mark_time_ranges=True)

In [8]:
files = glob.glob('docs/*.pdf')

In [14]:
ran = np.random.choice(500, 50)

In [15]:
ran

array([446, 383, 118, 413, 200, 207, 396, 122, 399, 362,  98, 495,  85,
        30,  58, 347, 114, 359, 120,   7, 146, 389, 480, 291, 365, 159,
       357,  31, 489, 153, 162,  43, 101,  34, 372, 464, 118, 271, 443,
       437, 190,  20, 291, 413, 133,  12,  91, 480, 217, 326])

In [16]:
try1 = [files[i] for i in ran]

In [17]:
try1

['docs/c4a9e9a8-4772-11e9-a06a-0242ac110002.pdf',
 'docs/1a6f4b30-4773-11e9-a06a-0242ac110002.pdf',
 'docs/1f46ad42-4773-11e9-a06a-0242ac110002.pdf',
 'docs/a5b46e88-4772-11e9-a06a-0242ac110002.pdf',
 'docs/eabbad3e-4772-11e9-a06a-0242ac110002.pdf',
 'docs/e50e9356-4772-11e9-a06a-0242ac110002.pdf',
 'docs/ad69c826-4772-11e9-a06a-0242ac110002.pdf',
 'docs/a6990886-4772-11e9-a06a-0242ac110002.pdf',
 'docs/d830587c-4772-11e9-a06a-0242ac110002.pdf',
 'docs/e22545fe-4772-11e9-a06a-0242ac110002.pdf',
 'docs/c4700350-4772-11e9-a06a-0242ac110002.pdf',
 'docs/cd27a66a-4772-11e9-a06a-0242ac110002.pdf',
 'docs/10c04134-4773-11e9-a06a-0242ac110002.pdf',
 'docs/b4121976-4772-11e9-a06a-0242ac110002.pdf',
 'docs/c66eb08e-4772-11e9-a06a-0242ac110002.pdf',
 'docs/a3e8b456-4772-11e9-a06a-0242ac110002.pdf',
 'docs/0c2e7df2-4773-11e9-a06a-0242ac110002.pdf',
 'docs/e36989d4-4772-11e9-a06a-0242ac110002.pdf',
 'docs/f973df22-4772-11e9-a06a-0242ac110002.pdf',
 'docs/96b2f85a-4772-11e9-a06a-0242ac110002.pdf',


In [18]:
time_output = {}
for file in try1:
#     print(file)
    fileID = file.split('/')[1].split('.')[0]
    with open(file, 'rb') as f:
        pages = pdftotext.PDF(f)
    text = ''
    for page in pages:
        text += page
    times = sutime.parse(text)
    candidates = []
    for time in times:
        if time['type'] == 'DATE' and re.search(r'(\d){4}-(\d){2}-(\d){2}', time['timex-value']):
            candidates.append(time['timex-value'])
        elif time['type'] == 'DURATION' and isinstance(time['value'], dict):
#             candidates.append(time['value']['end'])
            try:
                if re.search(r'(\d){4}-(\d){2}-(\d){2}', time['value']['end']):
                    time_output[fileID] = time['value']['end']
            except:
                continue
    candidates = sorted(candidates, reverse=True)
#     print(candidates)
    i = 0
    if not time_output.get(fileID):
        while candidates:
            if candidates[i].split('-')[1] != '02':
                if candidates[i].split('-')[-1] == '31' or candidates[i].split('-')[-1] == '30':
                    if len(candidates) > 1 and (candidates[i].split('-')[1]==candidates[i+1].split('-')[1] or int(candidates[i].split('-')[1])==int(candidates[i+1].split('-')[1])+1):
                        time_output[fileID] = candidates[i]
                    else:
                        time_output[fileID] = candidates[i]
                    break
                else:
                    candidates = candidates[i+1:]
                    i = 0
            else:
                if candidates[i].split('-')[-1] == '29' or candidates[i].split('-')[-1] == '28':
                    if len(candidates) > 1 and (candidates[i].split('-')[1]==candidates[i+1].split('-')[1] or int(candidates[i].split('-')[1])==int(candidates[i+1].split('-')[1])+1):
                        time_output[fileID] = candidates[i]
                    else:
                        time_output[fileID] = candidates[i]
                    break
                else:
                    candidates = candidates[i+1:]
                    i = 0
#             i += 1
#         for candidate in sorted(candidates)[::-1]:
#             if candidate.split('-'[1]) != 2:
#                 if candidate.split('-')[-1] == '31' or candidate.split('-')[-1] == '30':
#                     time_output[fileID] = candidate
#                     break
#                 else:
#                     candidate = candidate[:-1]

In [21]:
time_output

{'c4a9e9a8-4772-11e9-a06a-0242ac110002': '2014-11-30',
 '1a6f4b30-4773-11e9-a06a-0242ac110002': '2018-10-31',
 '1f46ad42-4773-11e9-a06a-0242ac110002': '2018-01-31',
 'a5b46e88-4772-11e9-a06a-0242ac110002': '2014-06-30',
 'eabbad3e-4772-11e9-a06a-0242ac110002': '2017-04-30',
 'e50e9356-4772-11e9-a06a-0242ac110002': '2017-08-31',
 'ad69c826-4772-11e9-a06a-0242ac110002': '2018-01-31',
 'a6990886-4772-11e9-a06a-0242ac110002': '2015-08-31',
 'd830587c-4772-11e9-a06a-0242ac110002': '2013-03-31',
 'e22545fe-4772-11e9-a06a-0242ac110002': '2017-09-30',
 'c4700350-4772-11e9-a06a-0242ac110002': '2015-09-30',
 'cd27a66a-4772-11e9-a06a-0242ac110002': '2015-04-30',
 '10c04134-4773-11e9-a06a-0242ac110002': '2017-09-30',
 'b4121976-4772-11e9-a06a-0242ac110002': '2018-06-30',
 'c66eb08e-4772-11e9-a06a-0242ac110002': '2013-12-31',
 'a3e8b456-4772-11e9-a06a-0242ac110002': '2014-02-28',
 '0c2e7df2-4773-11e9-a06a-0242ac110002': '2015-04-30',
 'e36989d4-4772-11e9-a06a-0242ac110002': '2015-03-31',
 'f973df22

In [22]:
fifty = [t.split('/')[1].split('.')[0] for t in try1]
fifty_time = [{rec['id']:rec['endingDate']} for rec in extracted_data if rec['id'] in fifty]

In [23]:
fifty_time

[{'96b2f85a-4772-11e9-a06a-0242ac110002': '05/31/2013'},
 {'98522000-4772-11e9-a06a-0242ac110002': '03/31/2017'},
 {'a006a4d8-4772-11e9-a06a-0242ac110002': 'APRIL 30, 2016'},
 {'a3e8b456-4772-11e9-a06a-0242ac110002': 'FEBRUARY 28, 2014'},
 {'a5b46e88-4772-11e9-a06a-0242ac110002': 'JUNE 30, 2014'},
 {'a6990886-4772-11e9-a06a-0242ac110002': 'August 31, 2015'},
 {'a8f90aea-4772-11e9-a06a-0242ac110002': '9/30/2018'},
 {'ac51db5e-4772-11e9-a06a-0242ac110002': 'APRIL 30, 2014'},
 {'ad69c826-4772-11e9-a06a-0242ac110002': 'January 31, 2018'},
 {'aebb026c-4772-11e9-a06a-0242ac110002': '9/30/2017'},
 {'b0e20996-4772-11e9-a06a-0242ac110002': '11/30/2014'},
 {'b4121976-4772-11e9-a06a-0242ac110002': '6/30/2018'},
 {'b5667948-4772-11e9-a06a-0242ac110002': 'January 31, 2018'},
 {'b6cf0192-4772-11e9-a06a-0242ac110002': 'May 31, 2013'},
 {'c067b6c2-4772-11e9-a06a-0242ac110002': '12/31/2015'},
 {'c2026d42-4772-11e9-a06a-0242ac110002': '05/31/2013'},
 {'c234e8da-4772-11e9-a06a-0242ac110002': 'June 30, 20

In [24]:
fifty_time_dict = {}
for time in fifty_time:
    fifty_time_dict[list(time.keys())[0]] = list(time.values())[0]

In [25]:
fifty_time_dict

{'96b2f85a-4772-11e9-a06a-0242ac110002': '05/31/2013',
 '98522000-4772-11e9-a06a-0242ac110002': '03/31/2017',
 'a006a4d8-4772-11e9-a06a-0242ac110002': 'APRIL 30, 2016',
 'a3e8b456-4772-11e9-a06a-0242ac110002': 'FEBRUARY 28, 2014',
 'a5b46e88-4772-11e9-a06a-0242ac110002': 'JUNE 30, 2014',
 'a6990886-4772-11e9-a06a-0242ac110002': 'August 31, 2015',
 'a8f90aea-4772-11e9-a06a-0242ac110002': '9/30/2018',
 'ac51db5e-4772-11e9-a06a-0242ac110002': 'APRIL 30, 2014',
 'ad69c826-4772-11e9-a06a-0242ac110002': 'January 31, 2018',
 'aebb026c-4772-11e9-a06a-0242ac110002': '9/30/2017',
 'b0e20996-4772-11e9-a06a-0242ac110002': '11/30/2014',
 'b4121976-4772-11e9-a06a-0242ac110002': '6/30/2018',
 'b5667948-4772-11e9-a06a-0242ac110002': 'January 31, 2018',
 'b6cf0192-4772-11e9-a06a-0242ac110002': 'May 31, 2013',
 'c067b6c2-4772-11e9-a06a-0242ac110002': '12/31/2015',
 'c2026d42-4772-11e9-a06a-0242ac110002': '05/31/2013',
 'c234e8da-4772-11e9-a06a-0242ac110002': 'June 30, 2018',
 'c4700350-4772-11e9-a06a-02

In [26]:
test_outcome = []
true_outcome = []
ids = time_output.keys()
for key in time_output.keys():
    test_outcome.append(time_output[key])
    true_outcome.append(fifty_time_dict[key])

In [31]:
time_df = pd.DataFrame([time_output.keys(), test_outcome, true_outcome], index=['file ID', 'date found', 'true date']).T

In [54]:
time_df

,file ID,date found,true date
0,c4a9e9a8-4772-11e9-a06a-0242ac110002,2014-11-30,"NOVEMBER 30, 2014"
1,1a6f4b30-4773-11e9-a06a-0242ac110002,2018-10-31,10/31/2018
2,1f46ad42-4773-11e9-a06a-0242ac110002,2018-01-31,"January 31, 2018"
3,a5b46e88-4772-11e9-a06a-0242ac110002,2014-06-30,"JUNE 30, 2014"
4,eabbad3e-4772-11e9-a06a-0242ac110002,2017-04-30,"April 30, 2017"
5,e50e9356-4772-11e9-a06a-0242ac110002,2017-08-31,08/31/2017
6,ad69c826-4772-11e9-a06a-0242ac110002,2018-01-31,"January 31, 2018"
7,a6990886-4772-11e9-a06a-0242ac110002,2015-08-31,"August 31, 2015"
8,d830587c-4772-11e9-a06a-0242ac110002,2013-03-31,"MARCH 31, 2013"
9,e22545fe-4772-11e9-a06a-0242ac110002,2017-09-30,"September 30, 2017"
